In [ ]:
import pathlib
import seaborn as sb
import networkx
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
from utils.paths import slugify
from utils.plotting import threshold_plot, plot_interval

## Declare parameters to use throughout

In [ ]:
events = [(24 + 12 * i, '', 'dashed') for i in range(4)]
phases = [
    (18, 24, 'N'),
    (24, 36, 'Day'),
    (36, 48, 'Night'),
    (48, 60, 'Day'),
    (60, 66, 'N')]
time_ticks = [18 + 6*i for i in range(8 + 1)]

output_directory = '../data/output'

### Load temporal data

In [ ]:
node_table_filepath = '../data/temporal_data/circadian/circadian_temporal_node_data_24_hour_genes_mean_normalised_full.csv'
static_network_filepath = '../data/static_networks/circadian_full.edgelist'

thresholds = [round(0 + (0.1 * i), 1) for i in range(6)]
combine_node_weights = lambda x, y: x * y

node_table = pd.read_csv(node_table_filepath, sep='\t', index_col=0)
static_network = networkx.read_edgelist(static_network_filepath, delimiter=', ')

edge_table = pd.DataFrame()
for i in node_table.columns.values:
    for j in node_table.columns.values:
        if i < j and static_network.has_edge(i, j):
            edge_table[f'({i}, {j})'] = combine_node_weights(node_table[i], node_table[j])

number_of_nodes = node_table.shape[1]
number_of_edges = edge_table.shape[1]
times = node_table.index.values

### Normalise

Comment out the code below to skip this step (e.g. if data is already normalised)

In [ ]:
normalise = None

if normalise == 'local':
    for node in node_table.columns.values:
        node_min = node_table[node].min()
        difference = node_table[node].max() - node_min
        node_table[node] = (node_table[node] - node_min) / difference

elif normalise == 'global':
    nodes_min = node_table.values.min()  # min(node_table[node].min() for node in node_table.columns.values)
    nodes_max = node_table.values.max()  # max(node_table[node].max() for node in node_table.columns.values)
    difference = nodes_max - nodes_min
    node_table = (node_table - nodes_min) / difference

    edges_min = edge_table.values.min()
    edges_max = edge_table.values.max()
    difference = edges_max - edges_min
    edge_table = (edge_table - edges_min) / difference

### Plot node activities

In [ ]:
fig, ax = plt.subplots(figsize=(9, 6))
cmap = cm.get_cmap('rainbow')

colour_interval = 1.0 / number_of_nodes
for j, node in enumerate(node_table.columns.values):
    color = cmap(colour_interval * j)
    ax.plot(times, node_table[node].values, color=color)

ax.set_ylabel("Gene expression")
ax.set_xlabel("Circadian time")
ax.set_xticks(time_ticks)

title = f'Node activities'
fig.suptitle(title, weight='bold')

if output_directory is not None:
    plt.savefig(f'{output_directory}/{title}.png', dpi=250)

### Plot edge activities

In [ ]:
fig, ax = plt.subplots(figsize=(9, 6))
cmap = cm.get_cmap('rainbow')

colour_interval = 1.0 / number_of_edges
for j, edge in enumerate(edge_table.columns.values):
    color = cmap(colour_interval * j)
    ax.plot(times, edge_table[edge].values, color=color)

ax.set_ylabel("Combined gene expression")
ax.set_xlabel("Circadian time")
ax.set_xticks(time_ticks)

title = f'Edge activities'
fig.suptitle(title)

if output_directory is not None:
    plt.savefig(f'{output_directory}/{title}.png', dpi=250)

### For each threshold, plot edge activities

In [ ]:
figsize = (9, 6*len(thresholds))
fig, axs = plt.subplots(len(thresholds), figsize=figsize)
cmap = cm.get_cmap('rainbow')

for ax, threshold in zip(axs, thresholds):
    colour_interval = 1.0 / number_of_edges

    for j, edge in enumerate(edge_table.columns.values):
        y = edge_table[edge].values
        color = cmap(colour_interval * j)

        if np.all(y >= threshold):
            ax.plot(times, y, color=color)
        elif np.all(y < threshold):
            ax.plot(times, y, color='k')
        else:
            threshold_plot(times, y, threshold, 'k', color, ax=ax)
        ax.axhline(threshold, color='k', ls='--')

    ax.set_ylim(0,1)
    ax.set_ylabel("Combined gene expression")
    ax.set_xlabel("Circadian time")
    ax.set_xticks(time_ticks)
    ax.set_title(f'Thresholded edges, threshold={threshold}', weight='bold')

title = f'Edge activities - {slugify(pathlib.Path(node_table_filepath).stem)}'
fig.suptitle(title, weight='bold', y=0.90)

if output_directory is not None:
    plt.savefig(f'{output_directory}/{title}.png', dpi=250)

### Plot the binarized edges

In [ ]:
edges = sorted(list(edge_table.columns))

for threshold in thresholds:
    fig, ax = plt.subplots(1, figsize=(15, 20))
    for j, edge in enumerate(edges):
        binarized_edges = ((edge_table[edge] > 0) & (edge_table[edge] >= threshold)).astype(int).values
        plot_interval(binarized_edges, times, y=number_of_edges-1-j, ax=ax, color='C0')

        # For readability, highlight one in every two lines
        if j % 2==0 :
            plot_interval(np.ones_like(binarized_edges), times, y=j, ax=ax, color='lightgrey', zorder=-3)

    ax.set_yticks([k + 0.25 for k in range(number_of_edges)])
    ax.set_yticklabels(reversed(edges), fontsize="xx-small")
    ax.set_ylim(0, number_of_edges)

    ax.set_xlim(min(times), max(times))
    ax.set_xticks(times)
    ax.set_xticklabels(times, rotation=45, ha='right', fontsize='x-small')
    ax.grid(axis='x')

    title = f'Binarised edges, threshold={threshold}'
    ax.set_title(title, weight='bold')
    sb.despine(left=True)

    if output_directory is not None:
        plt.savefig(f'{output_directory}/{title}.png', dpi=250)